worker ids:
46994197,CA,Redondo Beach, Salim, 21
47200615,NY,Ossining, Hee Dong, 21, 5
46545976,NY,Newburgh, Eva-Maria, 21, 3
46092070,NY,New City, Sara, 18, 2

In [ ]:
import pandas as pd
from datetime import datetime

name = {46994197:"Salim", 47200615:"Hee Dong", 46545976: "Eva-Maria", 46092070: "Sara"}
file_name = "/dccstor/srosent2/generative/appen/job_1988758NQLFQA.json"
data = pd.read_json(file_name, lines=True)

In [ ]:

skip_workers = {}
time_workers = {}
num_complete = {}

for worker in name:
    skip_workers[worker] = 0
    num_complete[worker] = 0
    # time_workers[worker] = 0

for index, row in data.iterrows():
    info = row['results']['judgments'][0]
    # print(row['results'])
        
    start = datetime.strptime(info['started_at'][11:-6],"%H:%M:%S")
    end = datetime.strptime(info['created_at'][11:-6],"%H:%M:%S")
    if info['worker_id'] not in time_workers:
        time_workers[info['worker_id']] = end-start
    else:
        time_workers[info['worker_id']] += end-start
    num_complete[info['worker_id']] += 1
    print(info['unit_data']['question_id']  + ": " + info['unit_data']['question'])
    print(name[info['worker_id']] + ": " + str(end-start))
    print("PARAGRAPH:")
    for sentence in info['unit_data']['long_answer']:
        print(sentence)
    if 'minimal_answer' in info['unit_data'] and info['unit_data']['minimal_answer'] is not None:
        print("minimal answer: " + info['unit_data']['minimal_answer'])
    if 'skipthere_is_no_answer' in info['data'] or 'there_is_no_answer' in info['data']:
        skip_workers[info['worker_id']] += 1
        print("SKIP")
    else:
        print("ANSWER: ")
        if 'paragraph_sentences' in info['data']:
            for sentence in info['data']['paragraph_sentences']:
                print(sentence)
        else:
            print("no sentences selected")
            print(info['data'])
        if 'type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_' not in info['data']:
        #    print(info['data']['type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_'])
        #else:
            print("no answer provided")
   
    print("---")
    # break
    # print(str(info['worker_id']) + "," + info['region'] + "," + info['city'] + "," + info['started_at'])
print(skip_workers)
for worker in name:
    print(name[worker] + ": " + str(time_workers[worker]/num_complete[worker]) + " skip: " +  str(skip_workers[worker]))

In [1]:
import pandas as pd
from datetime import datetime

name = {47200615:"Hee Dong", 46545976: "Eva Maria", 46092070: "Sara", 45676624: "Mohamed", 46373812: "Chie", 46545946: "Joekie", 46954475: "Arafat", 46994197: "Salim"}
file_name = "/dccstor/srosent2/generative/appen/job_2035917.json"
data = pd.read_json(file_name, lines=True)
data = data.sample(frac = 1)


In [2]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def rougeL(answer1, answer2):
    scores = scorer.score(answer1, answer2)
    return scores

In [ ]:

def process_data(data):

    avg_rouge_all =  {"Hee Dong": [], "Eva Maria": [], "Sara": [], "Mohamed": [], "Chie": [], "Joekie": [], "Arafat": []} 
    partial =  {"Hee Dong": 0, "Eva Maria": 0, "Sara": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0, "Arafat": 0} 
    
    for index, row in data.iterrows():
        avg_rouge = {"Hee Dong": [], "Eva Maria": [], "Sara": [], "Mohamed": [], "Chie": [], "Joekie": [], "Arafat": []}
        print(str(row['data']['question_id']) + ": " + row['data']['question'])
        print("Title: " + str(row['data']['title']))
        for sentence in row['data']['long_answer']:
            print(sentence)
        print("---")
        judgements = row['results']['judgments']
        skip = set()
        scores = {}

        i = 0
        for i in range(len(judgements)):
            answer_type = judgements[i]['data']['how_would_you_describe_the_questionanswer']

            if answer_type == 'complete' or answer_type == 'partial':
                if answer_type == 'partial':
                    partial[name[judgements[i]['worker_id']]] += 1
                # if 'type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_' not in judgements[i]['data']:
                #     print("Error!")
                #     print(judgements[i])
                #     continue
                # type_your_answer_here_keep_your_answer_as_close_to_the_passage_as_possible_
                answer1 = judgements[i]['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']
                print(name[judgements[i]['worker_id']] + ": " + answer1 + " (" + answer_type + ")")
                if 'comments_optional' in judgements[i]['data']:
                    print("comment: " + judgements[i]['data']['comments_optional'])
            else:
                skip.add(name[judgements[i]['worker_id']] + ": " + answer_type)
                continue

            for j in range(i+1,len(judgements)):
                answer_type = judgements[j]['data']['how_would_you_describe_the_questionanswer']

                if answer_type == 'complete' or answer_type == 'partial':
                    answer2 = judgements[j]['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']
                    # print(str(judgements[j]['worker_id']) + ": " + answer)
                else:
                    continue
                score = rougeL(answer1, answer2)
                if name[judgements[i]['worker_id']] == "Arafat" or name[judgements[i]['worker_id']] == "Sara" \
                     or name[judgements[j]['worker_id']] == "Arafat" or name[judgements[j]['worker_id']] == "Sara":
                    scores[name[judgements[i]['worker_id']] + "-" + name[judgements[j]['worker_id']]] = score['rougeL'].fmeasure
                    avg_rouge[name[judgements[i]['worker_id']]].append(score['rougeL'].fmeasure)
                    avg_rouge[name[judgements[j]['worker_id']]].append(score['rougeL'].fmeasure)
        print("skip: " + str(skip))
        scores = sorted(scores.items(), key=lambda item: item[1], reverse=True)

        for score in scores:
            print(score)

        print("Average: ")
        for worker in avg_rouge:
            if len(avg_rouge[worker]) > 0:
                avg = sum(avg_rouge[worker])/len(avg_rouge[worker])
                print(worker + ": " + str(avg))
                avg_rouge_all[worker].append(avg)
        print("-----------------")

        if index > 20:
            break
    print("Annotator| Avg RougeL|# answered")
    print("-- | -- | --")
    for worker in avg_rouge_all:
        avg = sum(avg_rouge_all[worker])/len(avg_rouge_all[worker])
        print(worker + "|" + str(avg) + "|" + str(len(avg_rouge_all[worker])))
        # print("partial count: " + worker + ": " + str(partial[worker]))

process_data(data)

This next code snippet is to analyze data when there is one annotation per example

In [7]:
import pandas
import numpy as np

def process_single_data(data):

    annotation_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    skip_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    sentence_count = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    sentences_per_paragraph = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    num_with_minimal_answer = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 

    # csvfile = open('/dccstor/srosent2/generative/appen/job_2035917.non_consecutive.xlsx', 'w', newline='')
    all_output = []

    for index, row in data.iterrows():
        judgement = row['results']['judgments'][0]

        # if len(row['data']['long_answer']) < 5:
        #     continue

        if judgement['unit_data']['minimal_text'] != '':
            num_with_minimal_answer[name[judgement['worker_id']]] += 1
            # continue
        
        answer_type = judgement['data']['how_would_you_describe_the_questionanswer']
        sentences_per_paragraph[name[judgement['worker_id']]] += len(row['data']['long_answer'])

        if answer_type == 'complete' or answer_type == 'partial':

            if 'type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_' not in judgement['data']:
                print(judgement)
                continue
            # if answer_type == 'partial':
            #     partial[name[judgement['worker_id']]] += 1
            if 'paragraph_sentences' in judgement['data']:
                sentence_count[name[judgement['worker_id']]] += len(judgement['data']['paragraph_sentences'])
            answer1 = judgement['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']

            last_consecutive = -1
            sent_i = 0
            non_consecutive = False
            for sentence in row['data']['long_answer']:
                if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
                    if last_consecutive != -1 and last_consecutive != sent_i-1:
                        non_consecutive = True
                        break
                    last_consecutive = sent_i
                sent_i += 1  

            if annotation_count[name[judgement['worker_id']]] < 10 and non_consecutive:
                output = [str(annotation_count[name[judgement['worker_id']]]) , str(row['data']['question_id']) , row['data']['question'], str(row['data']['title'])]
                
                sentences = ""
                for sentence in row['data']['long_answer']:
                    if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
                        sentences += "SELECTED " + sentence + "\n"
                    else:
                        sentences += sentence + "\n"
                output.append(sentences)
                output.append(name[judgement['worker_id']])
                output.append(answer1)
                output.append(answer_type)
                if 'comments_optional' in judgement['data']:
                    output.append(judgement['data']['comments_optional'])
                all_output.append(output)
                annotation_count[name[judgement['worker_id']]] += 1
        else:
            skip_count[name[judgement['worker_id']]] += 1
            continue
    with pd.ExcelWriter("/dccstor/srosent2/generative/appen/job_2035917.non_consecutive.xlsx", engine='xlsxwriter') as writer:
        df = pd.DataFrame(all_output)
        df.to_excel(writer, sheet_name='Sheet1')
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        cell_format = workbook.add_format({'text_wrap': True})
        worksheet.set_column('A:Z', cell_format=cell_format)
        
    # print("Annotator | # completed | # answered | # skipped | # with minimal answer | # sentences selected |  # sentences per answer | # sentences per paragraph")
    # print("-- | -- | -- | -- | -- | -- | -- | -- ")
    # for worker in annotation_count:
    #     print(worker + "|" + str(annotation_count[worker]) +  "|" +
    #       str(annotation_count[worker] - skip_count[worker]) + "|" + str(skip_count[worker]) + "|" + str(num_with_minimal_answer[worker]) + "|" + str(sentence_count[worker]) + "|" +
    #       str(round(sentence_count[worker]/(annotation_count[worker]-skip_count[worker]),2)) + "|" +  str(round(sentences_per_paragraph[worker]/(annotation_count[worker]-skip_count[worker]),2)))
    #     # print("partial count: " + worker + ": " + str(partial[worker]))

process_single_data(data)

Exception ignored in: <function ZipFile.__del__ at 0x7f63b6f18af0>
Traceback (most recent call last):
  File "/dccstor/srosent1/miniconda3/envs/primeqa4.24/lib/python3.9/zipfile.py", line 1816, in __del__
    self.close()
  File "/dccstor/srosent1/miniconda3/envs/primeqa4.24/lib/python3.9/zipfile.py", line 1833, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [4]:
import pandas
import numpy as np
import random

def appen_round2_check(data):

    annotation_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    skip_count =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    sentence_count = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    sentences_per_paragraph = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    num_with_minimal_answer = {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0} #, "Salim": 0} 
    new_task = {"Hee Dong": [], "Eva Maria": [], "Mohamed": [], "Chie": [], "Joekie": []}
    annotators = ["Hee Dong", "Eva Maria", "Mohamed", "Chie", "Joekie"]
    # csvfile = open('/dccstor/srosent2/generative/appen/job_2035917.non_consecutive.xlsx', 'w', newline='')

    annotated_df = pd.read_csv("/dccstor/srosent2/generative/appen/job_2035917.non_consecutive_annotated.csv", dtype=str)

    all_output = []

    for index, row in data.iterrows():

        if row['data']['question_id'] not in annotated_df['ID'].values:
            continue
        judgement = row['results']['judgments'][0]

        # if len(row['data']['long_answer']) < 5:
        #     continue

        if judgement['unit_data']['minimal_text'] != '':
            num_with_minimal_answer[name[judgement['worker_id']]] += 1
            # continue
        
        answer_type = judgement['data']['how_would_you_describe_the_questionanswer']
        sentences_per_paragraph[name[judgement['worker_id']]] += len(row['data']['long_answer'])

        if answer_type == 'complete' or answer_type == 'partial':

            if 'type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_' not in judgement['data']:
                print(judgement)
                continue
            # if answer_type == 'partial':
            #     partial[name[judgement['worker_id']]] += 1
            if 'paragraph_sentences' in judgement['data']:
                sentence_count[name[judgement['worker_id']]] += len(judgement['data']['paragraph_sentences'])
            answer1 = judgement['data']['type_your_answer_here_it_should_be_concise_and_only_come_from_the_passagetitle_']

            last_consecutive = -1
            sent_i = 0
            non_consecutive = False
            for sentence in row['data']['long_answer']:
                if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
                    if last_consecutive != -1 and last_consecutive != sent_i-1:
                        non_consecutive = True
                        break
                    last_consecutive = sent_i
                sent_i += 1  

            if non_consecutive:
                output = [str(row['data']['question_id']) , row['data']['question'], str(row['data']['title'])]
                
                sentences = ""
                selected = ""
                for sentence in row['data']['long_answer']:
                    sentences += sentence + "&nbsp;"
                    if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
                        selected += "True"
                    else:
                        selected += "False"
                    selected += "&nbsp;"
                title_used_in_answer = False
                if 'title_used_in_answer' in judgement['data']:
                    title_used_in_answer = True
                output.append(sentences)
                output.append(selected)
                output.append(title_used_in_answer)
                output.append(name[judgement['worker_id']])
                output.append(answer1)

                random_person = name[judgement['worker_id']]
                while random_person == name[judgement['worker_id']] or len(new_task[random_person]) == 10:
                    random_person = annotators[random.randint(0,len(annotators)-1)]
                print(random_person)
                new_task[random_person].append(output)
                annotation_count[name[judgement['worker_id']]] += 1
        else:
            skip_count[name[judgement['worker_id']]] += 1
            continue

    for person in new_task:
        df = pd.DataFrame(new_task[person])
        df.to_csv("/dccstor/srosent2/generative/appen/job_2035917.round2." + person + ".csv", encoding='utf-8', header=["id","question","title","long_answer","selected","title_used","worker","answer"])
        
    # print("Annotator | # completed | # answered | # skipped | # with minimal answer | # sentences selected |  # sentences per answer | # sentences per paragraph")
    # print("-- | -- | -- | -- | -- | -- | -- | -- ")
    # for worker in annotation_count:
    #     print(worker + "|" + str(annotation_count[worker]) +  "|" +
    #       str(annotation_count[worker] - skip_count[worker]) + "|" + str(skip_count[worker]) + "|" + str(num_with_minimal_answer[worker]) + "|" + str(sentence_count[worker]) + "|" +
    #       str(round(sentence_count[worker]/(annotation_count[worker]-skip_count[worker]),2)) + "|" +  str(round(sentences_per_paragraph[worker]/(annotation_count[worker]-skip_count[worker]),2)))
    #     # print("partial count: " + worker + ": " + str(partial[worker]))

appen_round2_check(data)

Mohamed
Hee Dong
Joekie
Mohamed
Eva Maria
Joekie
Mohamed
Eva Maria
Chie
Eva Maria
Hee Dong
Chie
Hee Dong
Chie
Mohamed
Mohamed
Hee Dong
Hee Dong
Mohamed
Joekie
Eva Maria
Joekie
Eva Maria
Hee Dong
Eva Maria
Joekie
Eva Maria
Chie
Hee Dong
Chie
Hee Dong
Chie
Joekie
Eva Maria
Mohamed
Hee Dong
Joekie
Hee Dong
Eva Maria
Chie
Mohamed
Eva Maria
Mohamed
Chie
Joekie
Mohamed
Joekie
Joekie
Chie
Chie


ValueError: Writing 8 cols but got 7 aliases

In [3]:
import pandas as pd
from datetime import datetime
import os

name = {47200615:"Hee Dong", 46545976: "Eva Maria", 45676624: "Mohamed", 46373812: "Chie", 46545946: "Joekie"}
dir_name = "/dccstor/srosent2/generative/appen/Round2/"
files = os.listdir(dir_name)

new_annotations = None
gold_annotations = None

for file_name in files:
    if file_name.endswith('json.zip'):
        if new_annotations is None:
            new_annotations = pd.read_json(dir_name + file_name, lines=True)
        else:
            new_annotations = new_annotations.append(pd.read_json(dir_name + file_name, lines=True))
    elif file_name.endswith('csv'):
        gold_annotations = pd.read_csv(dir_name + file_name, delimiter=',')

In [4]:

improvement =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0}
needs_improvement =  {"Hee Dong": 0, "Eva Maria": 0, "Mohamed": 0, "Chie": 0, "Joekie": 0}

all_output = []
for index, row in new_annotations.iterrows():
    output = []
    print(row['data']['question'])
    output.append(row['data']['id'])
    output.append(row['data']['question'])
    gold_annotation = gold_annotations[gold_annotations['ID'] == int(row['data']['id'])]

    sentences = ""
    judgement = row['results']['judgments'][0]
    input = row['data']
    for sentence in row['data']['long_answer']:
        selected = []
        if 'long_answer' in input and input['selected'][input['long_answer'].index(sentence)] == 'True':
            selected.append("1")
        if 'paragraph_sentences' in judgement['data'] and sentence in judgement['data']['paragraph_sentences']:
            selected.append("2")
        if len(selected) > 0:
            sentences += "SELECTED " + str(selected) + ": "
        sentences += sentence + "\n"

    answer1 = row['data']['answer']
    output.append(sentences)
    output.append(row['data']['worker'])
    output.append(answer1)
    print(row['data']['worker'] + ": " + answer1)
    output.append(name[row['results']['judgments'][0]['worker_id']])
    output.append(row['results']['judgments'][0]['data']['did_you_need_to_edit_the_answer'])
    if 'update_the_answer_here_if_needed' in row['results']['judgments'][0]['data']:
        answer2 = row['results']['judgments'][0]['data']['update_the_answer_here_if_needed']
        print(name[row['results']['judgments'][0]['worker_id']] + ": " + answer2)
        if 'did_you_need_to_edit_the_answer' in row['results']['judgments'][0]['data']:
            print(row['results']['judgments'][0]['data']['did_you_need_to_edit_the_answer'])
        rouge1 = rougeL(answer1, answer2)['rougeL'].fmeasure
        print(row['data']['worker'] + "/" + name[row['results']['judgments'][0]['worker_id']] + ": " + str(rouge1))
        output.append(answer2)
        output.append(rouge1)
    else:
        output.append("")
        output.append("")
    gold_answer = gold_annotation['Rewritten Answer'].iloc[0]
    output.append(gold_annotation['Assigned To'].iloc[0])
    if isinstance(gold_answer, str):
        needs_improvement[name[row['results']['judgments'][0]['worker_id']]] += 1
        print(gold_annotation['Assigned To'].iloc[0] + ": " + gold_answer)
        rouge2 = rougeL(answer2, gold_answer)['rougeL'].fmeasure
        rouge3 = rougeL(answer1, gold_answer)['rougeL'].fmeasure
        output.append(gold_answer)
        output.append(rouge2)
        output.append(rouge3)
        # if rouge3 > rouge2:
        improvement[name[row['results']['judgments'][0]['worker_id']]] += rouge3 - rouge2
        if rouge3 -.2 < rouge2:
            print("****")
        print(gold_annotation['Assigned To'].iloc[0] + "/" + row['data']['worker'] + ": " + str(rouge2))
        print(gold_annotation['Assigned To'].iloc[0] + "/" + name[row['results']['judgments'][0]['worker_id']] + ": " + str(rouge3))
    else:
        output.append("")
        output.append("")
        output.append("")
    print('--')
    all_output.append(output)

for annotator in improvement:
    if improvement[annotator] != 0:
        print(annotator + ": " + str(improvement[annotator]/needs_improvement[annotator]))
    else:
        print(annotator + ": 0/" + str(needs_improvement[annotator]))

with pd.ExcelWriter("/dccstor/srosent2/generative/appen/job_2035917.non_consecutive_followup_round2.xlsx", engine='xlsxwriter') as writer:
    df = pd.DataFrame(all_output)
    df.to_excel(writer, sheet_name='Sheet1')
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    cell_format = workbook.add_format({'text_wrap': True})
    worksheet.set_column('A:Z', cell_format=cell_format)

which pigment determines the colour of human hair
Mohamed: Eumelanin, which is behind the colors black , blond , and brown, and Pheomelanin, which is behind the red color, are the two pigments that determine the human hair color.
Joekie: Eumelanin determines the hair colors gray, black, blond and brown, and Pheomelanin, determines red hair color.
['extra info']
Mohamed/Joekie: 0.5454545454545454
--
when did we start putting shoes on horses
Hee Dong: Historians differ on the origin of the horseshoe. Because iron was a valuable commodity, and any worn out items were generally reforged and reused , it is difficult to locate clear archaeological evidence. In 1897 four bronze horseshoes with what are apparently nail holes were found in an Etruscan tomb dated around 400 B.C. The assertion by some historians that the Romans invented the `` mule shoes '' sometime after 100 BC is supported by a reference by Catullus who died in 54 BC.
Joekie: Historians differ on the origin of the horseshoe. Be